In [1]:
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd

In [2]:
BASE_PATH = './'

print('Reading data...')
loading_data = pd.read_csv(f'{BASE_PATH}/data/loading.csv')
trainscores_data = pd.read_csv(f'{BASE_PATH}/data/train_scores.csv')
fnc_data = pd.read_csv(f'{BASE_PATH}/data/fnc.csv')

Reading data...


In [3]:
fnc_features, loading_features = list(fnc_data.columns[1:]), list(loading_data.columns[1:])
df = fnc_data.merge(loading_data, on="Id")
trainscores_data["is_train"] = True
df = df.merge(trainscores_data, on="Id", how="left")

In [4]:
df

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),...,IC_30,IC_22,IC_29,IC_14,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,is_train
0,10001,0.368580,0.166876,0.438148,0.341007,-0.186251,0.049096,0.121417,-0.174268,-0.231578,...,0.002892,-0.023235,0.022177,0.017192,57.436077,30.571975,62.553736,53.325130,51.427998,True
1,10002,0.151696,-0.024819,0.217504,0.418072,-0.227234,-0.064052,-0.143832,-0.118116,-0.054825,...,0.002880,-0.016609,0.025543,0.014524,59.580851,50.969456,67.470628,60.651856,58.311361,True
2,10003,0.343415,0.109974,0.741641,0.578558,-0.676446,-0.436960,-0.295663,-0.377790,-0.344963,...,0.005211,-0.028882,0.031427,0.018164,NaN,NaN,NaN,NaN,NaN,NaN
3,10004,0.132793,0.258255,0.490769,0.342717,0.091112,0.107969,0.029220,-0.026237,0.094742,...,0.005866,-0.015182,0.024476,0.014760,71.413018,53.152498,58.012103,52.418389,62.536641,True
4,10005,0.291921,0.251254,0.416470,0.511719,-0.362626,-0.164710,-0.289059,-0.015537,-0.087316,...,0.002692,-0.019814,0.017105,0.013316,66.532630,NaN,NaN,52.108977,69.993075,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,21750,0.227970,-0.222489,0.250417,0.442642,-0.221094,-0.234935,-0.296614,-0.273216,-0.106481,...,0.007751,-0.020226,0.028821,0.017492,48.948756,55.114811,60.878271,38.617246,50.679885,True
11750,21751,0.455052,0.483856,0.589565,0.633691,0.161995,-0.175318,0.015480,0.135612,0.172296,...,0.001911,-0.008028,0.035093,0.017439,NaN,NaN,NaN,NaN,NaN,NaN
11751,21752,0.118257,0.452123,0.608328,0.422485,-0.106427,-0.008484,-0.010050,-0.122015,0.092994,...,0.001876,-0.014612,0.021665,0.019592,66.532630,59.844808,72.303110,55.458281,46.870235,True
11752,21753,0.051042,0.088581,0.551354,0.305542,-0.034378,-0.094778,-0.089954,-0.222876,-0.197503,...,0.003226,-0.003505,0.029326,0.014235,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
target_cols = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']
test_df = df[df["is_train"] != True].copy()
train_df = df[df["is_train"] == True].copy()

In [6]:
y_train_df = train_df[target_cols]
train_df = train_df.drop(target_cols + ['is_train'], axis=1)
test_df = test_df.drop(target_cols + ['is_train'], axis=1)

FNC_SCALE = 1/500
test_df[fnc_features] *= FNC_SCALE
train_df[fnc_features] *= FNC_SCALE

In [7]:
train_df

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),...,IC_08,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14
0,10001,0.000737,0.000334,0.000876,0.000682,-0.000373,0.000098,0.000243,-0.000349,-0.000463,...,0.018246,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192
1,10002,0.000303,-0.000050,0.000435,0.000836,-0.000454,-0.000128,-0.000288,-0.000236,-0.000110,...,0.014635,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524
3,10004,0.000266,0.000517,0.000982,0.000685,0.000182,0.000216,0.000058,-0.000052,0.000189,...,0.002658,0.022266,0.005956,-0.010595,0.024078,-0.000319,0.005866,-0.015182,0.024476,0.014760
4,10005,0.000584,0.000503,0.000833,0.001023,-0.000725,-0.000329,-0.000578,-0.000031,-0.000175,...,0.009702,0.017257,0.005454,-0.008591,0.019416,0.000786,0.002692,-0.019814,0.017105,0.013316
6,10007,0.000047,0.000504,0.001143,0.000677,-0.000209,-0.000399,-0.000482,-0.000382,-0.000057,...,0.006218,0.017332,0.016791,-0.009594,0.019757,0.003731,0.000733,-0.008462,0.026733,0.014358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11745,21746,0.000079,0.000156,0.000832,0.000874,-0.000235,0.000256,0.000124,-0.000409,-0.000201,...,0.010005,0.020480,0.011081,-0.015960,0.026506,0.006943,0.003312,-0.011562,0.032932,0.011053
11746,21747,0.000484,0.000417,0.001205,0.001266,-0.000267,-0.000428,-0.000785,-0.000450,-0.000775,...,0.013929,0.025991,0.010703,-0.010636,0.023988,0.001316,0.003792,-0.022357,0.031624,0.016982
11749,21750,0.000456,-0.000445,0.000501,0.000885,-0.000442,-0.000470,-0.000593,-0.000546,-0.000213,...,0.011454,0.022840,0.006448,-0.007203,0.029689,0.004942,0.007751,-0.020226,0.028821,0.017492
11751,21752,0.000237,0.000904,0.001217,0.000845,-0.000213,-0.000017,-0.000020,-0.000244,0.000186,...,0.012055,0.020741,0.001160,-0.007140,0.025236,0.002026,0.001876,-0.014612,0.021665,0.019592


In [12]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [13]:
param = {'objective':'regression',
        'metric':'rmse',
        'bossting_type':'gbdt',
        'learning_rate':0.01,
        'max_depth':-1}

output = pd.DataFrame()

for target in ['age','domain1_var1','domain1_var2','domain2_var1','domain2_var2']:
    
    X_train, X_val, y_train, y_val = train_test_split(train_df.iloc[:,1:], y_train_df[target], test_size=0.2, shuffle=True, random_state=20)
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)
    
    model = lgb.train(param, 
                      train_data, 
                      10000, 
                      early_stopping_rounds=15, 
                      valid_sets=[val_data], 
                      verbose_eval=50)
    
    temp = pd.DataFrame(test_df['Id'].apply(lambda x:str(x)+ '_'+ target))
    temp['Predicted'] = model.predict(test_df.iloc[:,1:])
    output = pd.concat([output,temp])

Training until validation scores don't improve for 15 rounds
[50]	valid_0's rmse: 11.978
[100]	valid_0's rmse: 11.1266
[150]	valid_0's rmse: 10.6695
[200]	valid_0's rmse: 10.4097
[250]	valid_0's rmse: 10.2486
[300]	valid_0's rmse: 10.1296
[350]	valid_0's rmse: 10.0424
[400]	valid_0's rmse: 9.98557
[450]	valid_0's rmse: 9.93042
[500]	valid_0's rmse: 9.88921
[550]	valid_0's rmse: 9.8579
[600]	valid_0's rmse: 9.83466
[650]	valid_0's rmse: 9.81733
[700]	valid_0's rmse: 9.7967
[750]	valid_0's rmse: 9.77993
[800]	valid_0's rmse: 9.76649
[850]	valid_0's rmse: 9.75487
[900]	valid_0's rmse: 9.74664
[950]	valid_0's rmse: 9.7333
[1000]	valid_0's rmse: 9.7284
[1050]	valid_0's rmse: 9.71884
[1100]	valid_0's rmse: 9.71331
[1150]	valid_0's rmse: 9.70829
[1200]	valid_0's rmse: 9.7011
[1250]	valid_0's rmse: 9.69817
[1300]	valid_0's rmse: 9.69354
Early stopping, best iteration is:
[1294]	valid_0's rmse: 9.69335
Training until validation scores don't improve for 15 rounds
[50]	valid_0's rmse: 17.6464
Ear